In [1]:
! nvidia-smi

Sat Apr  9 07:43:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#content to google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install tensorflow==2.2.0
!pip install keras==2.4.2
!pip install segmentation_models

     |████▎                           | 68.2 MB 2.0 MB/s eta 0:03:42

In [ ]:
# Enter modules
import zipfile  # For faster extraction
import keras
import tensorflow as tf
import os
from numpy import expand_dims
import cv2 # for read image
import numpy as np # For data manipulation
import matplotlib.pyplot as plt # for draw
from google.colab.patches import cv2_imshow #coordinate cv2 with colab
import nibabel as nib  #To read nii format
import re  # For parsing the filenames (to know their modality)
import glob # For populating the list of files
import imgaug.augmenters as iaa #for data agumention
from sklearn.model_selection import KFold, StratifiedKFold #Data setup and data segmentation
import imgaug as ia
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import time
import datetime
import os
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf
import segmentation_models as sm

In [ ]:
import os
import nibabel as nib
ADC=os.listdir('/content/drive/MyDrive/prostate_crop/ADC_crop')
print(len(ADC))

ADC_imag=[]
X=[]
for i in ADC:
  ADC_imag.append('/content/drive/MyDrive/prostate_crop/ADC_crop/' + i )
for j in ADC_imag:
  img= nib.load(j)
  adc_img = img.get_fdata(caching='unchanged') 
  X.append(adc_img)  

In [ ]:
label=os.listdir('/content/drive/MyDrive/prostate_crop/Label_crop')
print(len(label))
label_imag=[]
Y=[]

for i in label:
  label_imag.append('/content/drive/MyDrive/prostate_crop/Label_crop/' + i )
for j in label_imag:
  img= nib.load(j)
  lbl_img = img.get_fdata(caching='unchanged') 
  Y.append(lbl_img)  

In [ ]:
print(len(X),len(Y))

In [ ]:
X=np.array(X)
Y=np.array(Y)

In [ ]:
np.unique(Y[0])

In [ ]:
from sklearn.model_selection import KFold
i=0 
n_split=10
 
for train_index,test_index in KFold(n_split).split(X):
  if i==0:
    X_train,X_val=X[train_index],X[test_index]
    y_train,y_val=Y[train_index],Y[test_index]
  i=i+1  


print(len(X_train),len(y_train))
print(len(X_val),len(y_val))

In [ ]:
print(X_train.shape , y_train.shape ,X_val.shape, y_val.shape )

In [ ]:
X_T=[]
for i in range (0,len(X_train)):
  for j in range(0,19):
    img=X_train[i][:,:,j]
    X_T.append(img)

Y_T=[]
for i in range (0,len(y_train)):
  for j in range(0,19):
    img=y_train[i][:,:,j]
    Y_T.append(img)    

X_V=[]
for i in range (0,len(X_val)):
  for j in range(0,19):
    img=X_val[i][:,:,j]
    X_V.append(img) 
Y_V=[]
for i in range (0,len(y_val)):
  for j in range(0,19):
    img=y_val[i][:,:,j]
    Y_V.append(img)              

In [ ]:
np.unique(Y_V[0])

In [ ]:
print(len(X_T) , len(Y_T), len(X_V),len(Y_V))

In [ ]:
plt.imshow(Y_T[7])

In [ ]:
Y_T_c=to_categorical(Y_T,5)
Y_V_c=to_categorical(Y_V,5)

In [ ]:
np.unique(Y_T_c[0])

In [ ]:
X_T=np.array(X_T)
Y_T_c=np.array(Y_T_c)
X_V=np.array(X_V)
Y_V_c=np.array(Y_V_c)

In [ ]:
X_T=np.expand_dims(X_T, axis=-1)
X_V=np.expand_dims(X_V, axis=-1)

In [ ]:
print(X_T.shape , Y_T_c.shape ,X_V.shape, Y_V_c.shape )

In [ ]:
import random
def Data_generator_train(X,y, batch_size=8):   
    while True:               
          for start in range(0, X.shape[0], batch_size):
              x_batch = []
              y_batch = []
              end = min(start + batch_size, X.shape[0])
              ids_batch = X[start:end]
              for idd in range(start,end):           
                   a=random.randint(0,2)
                   if a==0:
                      image =np.fliplr(X[idd]) 
                      label =np.fliplr(y[idd])
                   if a==1:
                     image =np.flipud(X[idd]) 
                     label =np.flipud(y[idd])
                   elif a==2:
                     image =(X[idd]) 
                     label =(y[idd])                
                   x_batch.append(image)
                   y_batch.append(label)               
              x_batch= np.array(x_batch)
              y_batch= np.array(y_batch)                                           
              yield x_batch, y_batch


In [ ]:
m,b=next(iter(Data_generator_train(X_T,Y_T_c)))

print(m.shape)
print(b.shape)

In [ ]:
import random
def Data_generator_test(X,y, batch_size=1):   
    while True:               
          for start in range(0, X.shape[0], batch_size):
              x_batch = []
              y_batch = []
              end = min(start + batch_size, X.shape[0])
              ids_batch = X[start:end]
              for idd in range(start,end):           
                image =(X[idd]) 
                label =(y[idd])                
                x_batch.append(image)
                y_batch.append(label)               
              x_batch= np.array(x_batch)
              y_batch= np.array(y_batch)                                           
              yield x_batch, y_batch


In [ ]:
m,b=next(iter(Data_generator_test(X_V,Y_V_c)))

print(m.shape)
print(b.shape)

In [ ]:
model= sm.Unet('resnet34', classes=5, activation='softmax', encoder_weights=None, input_shape=(160, 160, 1))

In [ ]:
model.summary()

In [ ]:
optim = tf.keras.optimizers.Adam(lr =0.0008)
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy']

In [ ]:
model.compile(optim, total_loss, metrics)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger

model_checkpoint = ModelCheckpoint('/content/drive/My Drive/unet_resnet34_ADC/'+'unet_resnet34_fold1.h5', save_best_only=True,save_weights_only=True, verbose=2, monitor='val_f1-score',mode='max')
csv_logger = CSVLogger('/content/drive/My Drive/unet_resnet34_ADC/'+'unet_resnet34_fold1.csv', append=True)

In [ ]:
def step_decay(epoch):
     
     initAlpha = 0.0008
     factor = .9
     dropEvery = 30
     alpha = initAlpha * (factor ** np.floor((1 + epoch) / dropEvery))
     return float(alpha)

In [ ]:
batch_size =8
TrainSteps = len(X_T)  / batch_size
ValSteps   = len(X_V)

In [ ]:
# %%time
# from tensorflow.keras.callbacks import LearningRateScheduler

# history = model.fit_generator(Data_generator_train(X_T,Y_T_c), epochs = 50 , steps_per_epoch=TrainSteps, validation_data =Data_generator_test(X_V,Y_V_c) ,
#                 validation_steps =ValSteps,
#                callbacks=[model_checkpoint, csv_logger, LearningRateScheduler(step_decay)],shuffle=False)

In [ ]:
# plt.figure(figsize=(30, 5))
# plt.subplot(121)
# plt.plot(history.history['f1-score'])
# plt.plot(history.history['val_f1-score'])

# plt.title('Model Dice_score_unet_resnet34_fold1')
# plt.ylabel('dice_score')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')

# # Plot training & validation loss values
# plt.subplot(122)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss_unet_resnet34_fold1')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
# plt.figure(figsize=(30, 5))
# plt.subplot(121)
# plt.plot(history.history['iou_score'])
# plt.plot(history.history['val_iou_score'])

# plt.title('Model iou_score_unet_resnet34_fold1')
# plt.ylabel('iou_score')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')

# # Plot training & validation loss values
# plt.subplot(122)
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy_unet_resnet34_fold1')
# plt.ylabel('accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
model.load_weights('/content/drive/My Drive/unet_resnet34_ADC/unet_resnet34_fold1.h5')

In [ ]:
scores=model.evaluate(X_V,Y_V_c , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
p=model.predict(X_V)
y=Y_V_c

In [ ]:
from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,0], y[i,:,:,0])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

In [ ]:
from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,1], y[i,:,:,1])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

In [ ]:
from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,2], y[i,:,:,2])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

In [ ]:
from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,3], y[i,:,:,3])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

In [ ]:
from scipy.spatial.distance import directed_hausdorff


def Average(lst):
    return sum(lst) / len(lst)



HD=[]
for i in range(0,76):
  hd=directed_hausdorff(p[i,:,:,4], y[i,:,:,4])
  HD.append(hd[0])

print(len(HD))
print(Average(HD))

In [ ]:
pred=np.argmax(p, axis=-1)
ytrue=np.argmax(y, axis=-1)

In [ ]:
Pred=pred.flatten()
Ytrue=ytrue.flatten()

In [ ]:
Ytrue.shape

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Pred, Ytrue, digits=4))